# Calculating protected proportion in a grid
This notebook uses two feature layers each with a different type of protected areas: community vs non-community. 

In [1]:
import arcpy
import datetime
import arcgis
from arcgis.gis import GIS

In [ ]:
tmp = []
with open(".env") as f:
    for line in f:
        tmp.append(line)
aol_password = tmp[0].split("=")[1].split("\n")[0]
aol_username = tmp[1].split("=")[1].split("\n")[0]

In [ ]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)

In [2]:
arcpy.env.parallelProcessingFactor = "80%"
arcpy.env.workspace = "C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA\\WDPA.gdb"

In [3]:
mydate = datetime.datetime.now()
date = mydate.strftime("%y%b")
date

'20Jan'

In [ ]:
grid_id = "a52aa51088f84eb8b95f90be8fa77175"
grid_item=gis.content.get(grid_id)

In [4]:
# Initial feature collections needed
wdpa_not_comm_name = f"wdpa_not_comm_{date}"
wdpa_raisg_appended_name = f"comm_wdpa_raisg_appended_{date}"
mastergrid = grid_item.layers[0].url

In [5]:
# output names
wdpa_not_comm_dis = f"{wdpa_not_comm_name}_Dis"
wdpa_raisg_appended_dis = f"{wdpa_raisg_appended_name}_Dis"
wdpa_not_comm_in = f"{wdpa_not_comm_name}_In"
wdpa_raisg_appended_in = f"{wdpa_raisg_appended_name}_In"

### Not community WDPA
This layer has a lot of features and repair Geometry is necessary to run anything later

In [6]:
start = datetime.datetime.now()
arcpy.RepairGeometry_management(wdpa_not_comm_name)
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 3:45:56.846889


In [7]:
start = datetime.datetime.now()
arcpy.analysis.PairwiseDissolve(wdpa_not_comm_name, 
                                wdpa_not_comm_dis, 
                                None, None, "MULTI_PART")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:24:42.834690


In [8]:
start = datetime.datetime.now()
arcpy.analysis.PairwiseIntersect([wdpa_not_comm_dis, mastergrid], 
                                 wdpa_not_comm_in, 
                                 "ALL", None, "INPUT")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:40:07.911569


In [9]:
start = datetime.datetime.now()
arcpy.management.AddField(wdpa_not_comm_in, 
                          "AREA_KM2_prot", # the _prot is necessary because  biodiversity_facets has a field called AREA_KM2
                          "DOUBLE", 
                          None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:02.668914


In [10]:
start = datetime.datetime.now()
arcpy.management.CalculateGeometryAttributes(wdpa_not_comm_in, 
                                             "AREA_KM2_prot AREA_GEODESIC", 
                                             '', 
                                             "SQUARE_KILOMETERS", 
                                             "PROJCS['World_Cylindrical_Equal_Area',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Cylindrical_Equal_Area'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',0.0],PARAMETER['Standard_Parallel_1',0.0],UNIT['Meter',1.0]]")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

Finished in 1:20:12.690263


### There was an error, this is a logged bug, but the field was calculated.  
More information [here](https://community.esri.com/thread/237842-attributeerror-toolvalidator-object-has-no-attribute-islicensed-message) and the [reported bug](https://support.esri.com/en/bugs/nimbus/QlVHLTAwMDEyNDI4Ng==)


In [11]:
start = datetime.datetime.now()
arcpy.management.AddField(wdpa_not_comm_in, 
                          "not_comm_prot_prop", 
                          "DOUBLE", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:06.782340


In [12]:
start = datetime.datetime.now()
arcpy.management.CalculateField(wdpa_not_comm_in, 
                                "not_comm_prot_prop", 
                                "(!AREA_KM2_prot! /!AREA_KM2!) * 100", "PYTHON3", '')
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:36.916671


### Community WDPA
This layer has the WDPA community and the RAISG, repair geometry wasn't necesssary.

In [13]:
start = datetime.datetime.now()
arcpy.RepairGeometry_management(wdpa_raisg_appended_name)
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 4:51:54.130626


In [14]:
start = datetime.datetime.now()
arcpy.analysis.PairwiseDissolve(wdpa_raisg_appended_name, 
                                wdpa_raisg_appended_dis)
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:09:06.136200


In [15]:
start = datetime.datetime.now()
arcpy.analysis.PairwiseIntersect([wdpa_raisg_appended_dis, mastergrid], 
                                 wdpa_raisg_appended_in, 
                                 "ALL", None, "INPUT")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:17:23.811639


In [16]:
start = datetime.datetime.now()
arcpy.management.AddField(wdpa_raisg_appended_in, 
                          "AREA_KM2_prot", 
                          "DOUBLE", 
                          None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:02.433103


In [17]:
start = datetime.datetime.now()
arcpy.management.CalculateGeometryAttributes(wdpa_raisg_appended_in, 
                                             "AREA_KM2_prot AREA", '', 
                                             "SQUARE_KILOMETERS", 
                                             "PROJCS['World_Cylindrical_Equal_Area',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Cylindrical_Equal_Area'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',0.0],PARAMETER['Standard_Parallel_1',0.0],UNIT['Meter',1.0]]")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

Finished in 0:13:00.537500


In [18]:
start = datetime.datetime.now()
arcpy.management.AddField(wdpa_raisg_appended_in, 
                          "comm_prot_prop", 
                          "DOUBLE", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:04.592219


In [19]:
start = datetime.datetime.now()
arcpy.management.CalculateField(wdpa_raisg_appended_in, 
                                "comm_prot_prop",
                                "(!AREA_KM2_prot! /!AREA_KM2!) * 100", "PYTHON3", '')
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:09.711241


### Merging all Community and non-Community WDPA
This is necessary to calculate the proportion protected by any type of protected area. In some parts of the world community and non-community protection overlap. 

In [20]:
intersects_merge = f"Intersects_Merge{date}"

In [21]:
start = datetime.datetime.now()
arcpy.management.Merge([wdpa_not_comm_in, wdpa_raisg_appended_in],
                       intersects_merge)
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:01:28.038105


In [22]:
intersects_merge_dis = f"{intersects_merge}_Dis"
start = datetime.datetime.now()
arcpy.analysis.PairwiseDissolve(intersects_merge, 
                                intersects_merge_dis, 
                                "CELL_ID", "AREA_KM2 MAX", "MULTI_PART")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:57:07.739189


In [23]:
for field in arcpy.ListFields(intersects_merge_dis):
    print(field.name)

OBJECTID
Shape
CELL_ID
MAX_AREA_KM2
Shape_Length
Shape_Area


In [24]:
start = datetime.datetime.now()
arcpy.management.AddField(intersects_merge_dis, 
                          "AREA_KM2_prot", 
                          "DOUBLE", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:02.964169


In [25]:
start = datetime.datetime.now()
arcpy.management.CalculateGeometryAttributes(intersects_merge_dis,                                               
                                             "AREA_KM2_prot AREA_GEODESIC", '', 
                                             "SQUARE_KILOMETERS", 
                                             "PROJCS['World_Cylindrical_Equal_Area',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Cylindrical_Equal_Area'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',0.0],PARAMETER['Standard_Parallel_1',0.0],UNIT['Meter',1.0]]")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

Finished in 1:52:39.247712


In [26]:
start = datetime.datetime.now()
arcpy.management.AddField(intersects_merge_dis, 
                          "all_prot_prop", 
                          "DOUBLE", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:03.097969


In [27]:
start = datetime.datetime.now()
arcpy.management.CalculateField(intersects_merge_dis, 
                                "all_prot_prop",  
                                "(!AREA_KM2_prot! / !MAX_AREA_KM2!) * 100", "PYTHON3", '')
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:48.334536


### Getting the information from the 3 tables into one

The geoprocessing tool to use is Join Field, which should be run three times with the `cellID_areaKM2` layer always as input.

In [28]:
start = datetime.datetime.now()
arcpy.management.JoinField(intersects_merge_dis, 
                           "CELL_ID", 
                           wdpa_not_comm_in, 
                           "CELL_ID", 
                           "not_comm_prot_prop")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:50.098308


In [29]:
start = datetime.datetime.now()
arcpy.management.JoinField(intersects_merge_dis, 
                           "CELL_ID", 
                           wdpa_raisg_appended_in, 
                           "CELL_ID", 
                           "comm_prot_prop")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:26.934896


After joining there will be a lot of `null` values that have to be changed into zeroes. [Here](https://support.esri.com/en/technical-article/000016100) there is a nice techinical article explaining how to do it. `Null` in arcgis is 
```python
None
```

In [30]:
fieldObs = arcpy.ListFields(intersects_merge_dis)  
fieldNames = []
for field in fieldObs:
    fieldNames.append(field.name)
del fieldObs
fieldCount = len(fieldNames)

In [31]:
start = datetime.datetime.now()
with arcpy.da.UpdateCursor(intersects_merge_dis, fieldNames) as curU:  
    for row in curU:  
        rowU = row 
        break
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0"  
        curU.updateRow(rowU)
del curU
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:00.286011


The table has to be exported as .csv (use `Table to table`) and then uploaded to the arcgis account see [here](https://developers.arcgis.com/labs/python/import-data/)
Then it has to be "related". The `Join` within arcgis online does not work. 
To create a feature layer that contains related records, you must publish the layer from ArcGIS Pro or ArcMap.

In [32]:
start = datetime.datetime.now()
csv_name = f"prot_prop_{date}.csv"
arcpy.conversion.TableToTable(intersects_merge_dis, 
                              r"C:\Users\Vizuser\Documents\ArcGIS\Projects\WDPA", 
                              csv_name)
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:16.492217
